In [1]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta, adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils.vis_utils  import plot_model
from sklearn import preprocessing
from keras import optimizers

print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Using TensorFlow backend.


Keras version: 		2.0.9
Scikit version: 	0.19.1
TensorFlow version: 	1.4.1


In [2]:
import nltk
from nltk.corpus import stopwords
from tika import parser
from sklearn.metrics import classification_report



In [3]:
def LoadData_tech():
    label_tech=[]
    i=0
    for foldername in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/tech"):
        for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/tech/"+foldername):
    
            try:
                print (i, foldername, file)
                parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/tech/"+foldername+"/"+file)

                resume_contents=clean_doc(parsedPDF['content'])
            #Data = resume_contents.encode('utf-8')    
                label_tech.append((resume_contents,foldername))
            except UnicodeEncodeError:
                print ('Unicode error:', file)
            i=i+1
    #print (label_resume)
    return(label_tech)
        

In [5]:
#Preprocessing
def clean_doc(doc):
    """
    Cleaning a document by several methods:
        - Lowercase
        - Removing whitespaces
        - Removing stopwords
        - Removing punctuations
        
    """
    stop_words = set(stopwords.words('english'))
    
    # Lowercase
    doc = doc.lower()
   
    tokens = doc.translate({ord(c):"" for c in "\u200b\uf020\u2028\xa0\uf0e0\uf095\uf041\uf0e1\uf0b7\xad"})
    tokens = tokens.translate({ord(c):" " for c in "[):,·](;•●■♦▪"})
    tokens = tokens.translate({ord(c):"f" for c in "�"})
    
    # Removing multiple whitespaces
    tokens = re.sub(r"\?", " \? ", tokens)
    
    # Split in tokens
    tokens = tokens.split()
    
    # Remove Stopwords
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuation
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    
    return ' '.join(tokens)

In [5]:
label_tech=LoadData_tech()

0 content writer BrettJohnsonProfile.pdf
1 content writer Maria ChristineJohnsonProfile.pdf
2 content writer HelenHarbord FISTCProfile.pdf
3 content writer MohanReddy[7_0].doc
4 content writer PiyushPranavProfile.pdf
5 content writer Content Writer -Interested.docx
6 content writer CV_Vibin.docx
7 content writer Prem Hans RajSinghProfile.pdf
8 content writer AnjuRachelThomas[1_0].docx
9 content writer Febajacob[2_0].docx
10 content writer ManjunathMProfile.pdf
11 content writer PreetiKusnoorProfile.pdf
12 content writer NandiniVijayagopal[7_0].docx
13 content writer NandlalSharma[5_4].doc
14 content writer NityaMarwah[5_0].docx
15 content writer AarushiNairProfile.pdf
16 content writer AmelAbdul RahmanProfile.pdf
17 content writer DheerajKapoor[5_0].pdf
18 content writer NatashaMirandaProfile.pdf
19 content writer DivyaBhanotProfile.pdf
20 content writer RadhikaLingampalliProfile.pdf
21 content writer Deepthi.pdf
22 content writer AJAYKUMARM[5_2].docx
23 content writer ABHIJITHT[2_0].d

194 IT 15918607_Noida_9.05_yrs.doc
195 IT amanulla_syed_Hyderabad___Secunderabad_0.03_yrs.docx
196 IT 83886787_Hyderabad___Secunderabad_3.01_yrs.pdf
197 IT Chetan_Sangule_Pune_3.00_yrs.docx
198 IT HamedAhmadzadehProfile.pdf
199 IT 79983544_Pune_1.04_yrs.pdf
200 IT Basil Joseph.pdf
201 IT AvinashGowdaProfile.pdf
202 IT CCNA-Network-Engineer-Resume-Free-PDF-Download.pdf
203 IT JesseStanfordProfile.pdf
204 IT 36968539_Bengaluru___Bangalore_12.00_yrs.docx
205 IT Ashish_Wankhede_Pune_1.08_yrs.docx
206 IT jothi_prakash_Bengaluru___Bangalore_8.05_yrs.doc
207 IT IT Network Administrator Resume.pdf
208 IT HaniGorishProfile.pdf
209 IT KhalidMehmoodProfile.pdf
210 IT BahadorGheisari(B.Eng,PMP,CCNP)Profile.pdf
211 IT CCNP-Network-Engineer-Resume-Free-Word-Download.doc
212 IT MarcelleLeeProfile.pdf
213 IT AmitSinghProfile.pdf
214 IT DEEPAK_JATHAN_Mumbai_City_10.00_yrs.docx
215 IT Carol RogerBProfile.pdf
216 IT AJAY_RESUME (2).pdf
217 IT Deepak_Pandiyan_Bengaluru___Bangalore_5.00_yrs.doc
218 IT LeeC

379 software Harika_Golla_Hyderabad___Secunderabad_2.00_yrs.docx
380 software Manicka_Govinda_Ragul.M___Chennai_3.02_yrs.docx
381 software Aamir MohammedKhanProfile.pdf
382 software SHINTUV[3_0].docx
383 software Dipin.K.doc
384 software SergiiOstapenkoProfile.pdf
385 software SREEKALACS[3_1].docx
386 software JESMI A M.doc
387 software Gaurav_Balyan_Noida_4.11_yrs.docx
388 software priyanka_reddy_Hyderabad___Secunderabad_3.02_yrs.docx
389 software ArghyaBiswas ( ETL - QA)Profile.pdf
390 software Haritha_Madhavi_Hyderabad___Secunderabad_7.00_yrs.docx
391 software lavanya_methuku_Bengaluru___Bangalore_2.09_yrs.docx
392 software Soumyaranjan[2_5].docx
393 software Charan DasNainiProfile.pdf
394 software 53672745_Bengaluru___Bangalore_2.05_yrs.docx
395 software PallaviDubeyProfile.pdf
396 software Sagar[7_0].docx
397 software MidhunsudhakarProfile.pdf
398 software ANUSHAK[3_1].pdf
399 software AbdulMoizProfile.pdf
400 software Nikhil__Pune_3.00_yrs.doc
401 software Naga_Gayathri_Devi_Hyde

572 others Mariselvam_M_G__Coimbatore_0.04_yrs.docx
573 others Anumol_Abraham_Cochin___Kochi___Ernakulam_0.07_yrs.pdf
574 others GiuliaSonninoProfile(1).pdf
575 others martamunte vidalProfile.pdf
576 others Gokulrah__Chennai_1.00_yrs.pdf
577 others AndiB.Profile.pdf
578 others ClaudiaBrambillaProfile.pdf
579 others kamna_pandey_Delhi_0.00_yrs.pdf
580 others Alfina_Muthu_Chennai_1.05_yrs.docx
581 others Akansha_Singhal_Noida_2.00_yrs.docx
582 others 49700435_Bengaluru___Bangalore_16.00_yrs.doc
583 others gireeshguptaProfile.pdf
584 others Jitendra_Prajapati_Delhi_6.08_yrs.docx
585 others EmmaAllsupProfile.pdf
586 others Dharamendra_Singh_Delhi_5.02_yrs.doc
587 others Amir_Dhuri_Pune_2.08_yrs.docx
588 others anilvishnu1(sh)15thJan'16.docx
589 others Deepak_Chandel_Delhi_6.00_yrs.doc
590 others NURULHUDAProfile.pdf
591 others AndieGalloProfile.pdf
592 others Animator Fresher Resume.pdf
593 others ElisaHartikkaProfile.pdf
594 others 3D Animator Resume.doc
595 others AngelicaChieffiProfile.

In [6]:
len(label_tech)

600

In [7]:

jdres_train_labels=[]
jdres_train_data=[]

for row in label_tech:
    jdres_train_data.append(row[0])
    jdres_train_labels.append(row[1])
#print(jdres_train_labels)

#print(len(jdres_train_data))

In [8]:
print(len(jdres_train_data))
print(len(jdres_train_labels))

600
600


In [9]:
#print(jdres_train_data[42])

In [10]:
encoder = preprocessing.LabelEncoder()
encoder.fit(jdres_train_labels)
encoded_labels = encoder.transform(jdres_train_labels)
# convert integers to dummy variables (i.e. one hot encoded)
labels = to_categorical(encoded_labels,num_classes=4)
print(encoded_labels)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [11]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(jdres_train_data)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(jdres_train_data)
vocab_size

27872

In [12]:
# pad documents to a max length of 50 words
max_length = 1000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[   21    19  6481 ...,     0     0     0]
 [   21    19  5397 ...,     0     0     0]
 [   21    19  6498 ...,     0     0     0]
 ..., 
 [  482    67 12512 ...,     0     0     0]
 [   21    19  4184 ...,     0     0     0]
 [12520   595     1 ...,     0     0     0]]


In [13]:
print('Shape of data tensor:', padded_docs.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (600, 1000)
Shape of label tensor: (600, 4)


In [14]:
indices = np.arange(padded_docs.shape[0])
np.random.shuffle(indices)
padded_docs = padded_docs[indices]
labels = labels[indices]

In [15]:
#loading glove

embeddings_index = dict()
f = open('/home/shabna/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [16]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [17]:
#create model

from keras.layers import Activation, Input, Dense, Flatten, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D,GlobalMaxPooling1D
from keras.models import Model

In [18]:
#from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [19]:
def create_model():
    model = Sequential()
    e = Embedding(vocab_size,100, weights=[embedding_matrix], input_length=1000, trainable='true')
    model.add(e)
    model.add(Conv1D(128,5, activation='relu',name='l1'))
    model.add(MaxPooling1D(pool_size=5,name='l2'))
    
    model.add(Conv1D(128, 5, activation='relu',name='l3'))
    model.add(GlobalMaxPooling1D(name='l4'))
   
    model.add(Dense(12, activation='sigmoid',name='l5'))
    model.add(Dropout(0.2))
    #model.add(Flatten())
    model.add(Dense(4, activation='softmax',name='l6'))
    #adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [20]:
#model = Model(embedding_matrix, preds)
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         2787200   
_________________________________________________________________
l1 (Conv1D)                  (None, 996, 128)          64128     
_________________________________________________________________
l2 (MaxPooling1D)            (None, 199, 128)          0         
_________________________________________________________________
l3 (Conv1D)                  (None, 195, 128)          82048     
_________________________________________________________________
l4 (GlobalMaxPooling1D)      (None, 128)               0         
_________________________________________________________________
l5 (Dense)                   (None, 12)                1548      
_________________________________________________________________
dropout_1 (Dropout)          (None, 12)                0         
__________

In [ ]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(learn_rate=learn_rate, momentum=momentum, batch_size=batch_size, epochs=epochs,optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result = grid.fit(X_train, y_train,validation_data=(X_val,y_val))
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [21]:
from keras.callbacks import ModelCheckpoint

In [22]:
from keras.callbacks import Callback

In [23]:
filepath="/home/shabna/Desktop/example_codes/weights_3_1.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only="True", mode="max")
callbacks_list = [checkpoint]

In [24]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

XX_train, X_test, yy_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=42)

i=1
for train_index, test_index in kf.split(XX_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    print(i,":num of fold")

    X_train, X_val = XX_train[train_index], XX_train[test_index]
    y_train, y_val = yy_train[train_index], yy_train[test_index]
    
    history = model.fit(X_train, y_train, validation_data=(X_val,y_val) ,epochs=50, batch_size=100, verbose=1, callbacks=callbacks_list)
   
    i=i+1
    

1 :num of fold
Train on 384 samples, validate on 96 samples
Epoch 1/50
384/384 [==============================] - 32s 84ms/step - loss: 1.4869 - acc: 0.2344 - val_loss: 1.3961 - val_acc: 0.2396
Epoch 2/50
384/384 [==============================] - 21s 56ms/step - loss: 1.3851 - acc: 0.2839 - val_loss: 1.3806 - val_acc: 0.2604
Epoch 3/50
384/384 [==============================] - 22s 56ms/step - loss: 1.3592 - acc: 0.2995 - val_loss: 1.3420 - val_acc: 0.6042
Epoch 4/50
384/384 [==============================] - 19s 48ms/step - loss: 1.3125 - acc: 0.4635 - val_loss: 1.3282 - val_acc: 0.2604
Epoch 5/50
384/384 [==============================] - 19s 49ms/step - loss: 1.2829 - acc: 0.3984 - val_loss: 1.2944 - val_acc: 0.4792
Epoch 6/50
384/384 [==============================] - 24s 63ms/step - loss: 1.2335 - acc: 0.5755 - val_loss: 1.2569 - val_acc: 0.6458
Epoch 7/50
384/384 [==============================] - 43s 111ms/step - loss: 1.2016 - acc: 0.5833 - val_loss: 1.2232 - val_acc: 0.6458
E

Epoch 29/50
384/384 [==============================] - 30s 78ms/step - loss: 0.4911 - acc: 0.9740 - val_loss: 0.5347 - val_acc: 0.9583
Epoch 30/50
384/384 [==============================] - 30s 78ms/step - loss: 0.4695 - acc: 0.9896 - val_loss: 0.5189 - val_acc: 0.9583
Epoch 31/50
384/384 [==============================] - 30s 79ms/step - loss: 0.4473 - acc: 0.9740 - val_loss: 0.5112 - val_acc: 0.9583
Epoch 32/50
384/384 [==============================] - 30s 79ms/step - loss: 0.4479 - acc: 0.9818 - val_loss: 0.5023 - val_acc: 0.9583
Epoch 33/50
384/384 [==============================] - 30s 78ms/step - loss: 0.4424 - acc: 0.9818 - val_loss: 0.4861 - val_acc: 0.9583
Epoch 34/50
384/384 [==============================] - 30s 78ms/step - loss: 0.4271 - acc: 0.9844 - val_loss: 0.4795 - val_acc: 0.9583
Epoch 35/50
384/384 [==============================] - 30s 78ms/step - loss: 0.4121 - acc: 0.9922 - val_loss: 0.4671 - val_acc: 0.9583
Epoch 36/50
384/384 [==============================] - 

Epoch 11/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2865 - acc: 0.9870 - val_loss: 0.2223 - val_acc: 1.0000
Epoch 12/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2813 - acc: 0.9792 - val_loss: 0.2184 - val_acc: 1.0000
Epoch 13/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2842 - acc: 0.9687 - val_loss: 0.2153 - val_acc: 1.0000
Epoch 14/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2905 - acc: 0.9818 - val_loss: 0.2121 - val_acc: 1.0000
Epoch 15/50
384/384 [==============================] - 30s 77ms/step - loss: 0.2649 - acc: 0.9948 - val_loss: 0.2093 - val_acc: 1.0000
Epoch 16/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2726 - acc: 0.9870 - val_loss: 0.2063 - val_acc: 1.0000
Epoch 17/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2555 - acc: 0.9844 - val_loss: 0.2033 - val_acc: 1.0000
Epoch 18/50
384/384 [==============================] - 

Epoch 44/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2149 - acc: 0.9922 - val_loss: 0.1435 - val_acc: 1.0000
Epoch 45/50
384/384 [==============================] - 30s 79ms/step - loss: 0.1831 - acc: 0.9896 - val_loss: 0.1416 - val_acc: 1.0000
Epoch 46/50
384/384 [==============================] - 30s 77ms/step - loss: 0.1921 - acc: 0.9844 - val_loss: 0.1404 - val_acc: 1.0000
Epoch 47/50
384/384 [==============================] - 30s 77ms/step - loss: 0.1936 - acc: 0.9844 - val_loss: 0.1389 - val_acc: 1.0000
Epoch 48/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2042 - acc: 0.9948 - val_loss: 0.1365 - val_acc: 1.0000
Epoch 49/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1985 - acc: 0.9844 - val_loss: 0.1348 - val_acc: 1.0000
Epoch 50/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1999 - acc: 0.9766 - val_loss: 0.1333 - val_acc: 1.0000
3 :num of fold
Train on 384 samples, validate on 96 sam

384/384 [==============================] - 30s 79ms/step - loss: 0.1292 - acc: 0.9948 - val_loss: 0.0712 - val_acc: 1.0000
Epoch 9/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1299 - acc: 0.9896 - val_loss: 0.0706 - val_acc: 1.0000
Epoch 10/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1208 - acc: 0.9818 - val_loss: 0.0699 - val_acc: 1.0000
Epoch 11/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1332 - acc: 0.9818 - val_loss: 0.0693 - val_acc: 1.0000
Epoch 12/50
384/384 [==============================] - 30s 79ms/step - loss: 0.1256 - acc: 0.9844 - val_loss: 0.0687 - val_acc: 1.0000
Epoch 13/50
384/384 [==============================] - 30s 79ms/step - loss: 0.1307 - acc: 0.9688 - val_loss: 0.0682 - val_acc: 1.0000
Epoch 14/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1317 - acc: 0.9844 - val_loss: 0.0676 - val_acc: 1.0000
Epoch 15/50
384/384 [==============================] - 30s 78ms/step

Epoch 41/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1110 - acc: 0.9844 - val_loss: 0.0539 - val_acc: 1.0000
Epoch 42/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1136 - acc: 0.9870 - val_loss: 0.0535 - val_acc: 1.0000
Epoch 43/50
384/384 [==============================] - 30s 79ms/step - loss: 0.1109 - acc: 0.9896 - val_loss: 0.0530 - val_acc: 1.0000
Epoch 44/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1163 - acc: 0.9844 - val_loss: 0.0526 - val_acc: 1.0000
Epoch 45/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1157 - acc: 0.9766 - val_loss: 0.0522 - val_acc: 1.0000
Epoch 46/50
384/384 [==============================] - 30s 77ms/step - loss: 0.1214 - acc: 0.9844 - val_loss: 0.0517 - val_acc: 1.0000
Epoch 47/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1187 - acc: 0.9818 - val_loss: 0.0513 - val_acc: 1.0000
Epoch 48/50
384/384 [==============================] - 

In [3]:
model=load_model("weights_3_1.best.hdf5")


In [31]:
plot_model(model, to_file = 'models.png')

In [26]:
y_train

array([[ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.]])

In [27]:
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred,axis=1)

In [29]:
y_pred = model.predict_classes(X_test)

120/120 [==============================] - 3s 22ms/step


In [30]:
p = model.predict_proba(X_test)

120/120 [==============================] - 3s 23ms/step


In [31]:
model.evaluate(X_test,y_test)

120/120 [==============================] - 3s 25ms/step


[0.33808534145355223, 0.96666667064030964]

In [32]:
model.evaluate(X_train,y_train)

384/384 [==============================] - 9s 22ms/step


[0.27982838203509647, 0.9921875]

In [33]:
labels_index = {'IT':0 ,'contentwriter':1, 'other':2, 'software':3}
target_name = [t for t in labels_index.keys()]

In [34]:
print(classification_report(np.argmax(y_test,axis=1),y_pred,target_names=target_name))

               precision    recall  f1-score   support

           IT       1.00      0.97      0.98        31
contentwriter       1.00      0.96      0.98        28
        other       1.00      0.95      0.97        37
     software       0.86      1.00      0.92        24

  avg / total       0.97      0.97      0.97       120



In [35]:
print(confusion_matrix(np.argmax(y_test,axis=1),y_pred))

[[30  0  0  1]
 [ 0 27  0  1]
 [ 0  0 35  2]
 [ 0  0  0 24]]


In [155]:
from numpy import argmax
for i, item in enumerate (X_test):
    item= item.reshape(1, 1000)
    #print (item.shape)
    classes=model.predict(item)
    p=(argmax(classes),argmax(y_test[i]))
    #print(p)
    if (argmax(classes)!=argmax(y_test[i])):
        print(p)


(3, 0)
(3, 1)
(3, 2)
(3, 2)


In [36]:
#print(classification_report(np.argmax(y_train,axis=1),y_pred1,target_names=target_name))

In [6]:
#predictions

parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/tech/IT/Arju_Utpal__Bengaluru___Bangalore_0.00_yrs.pdf")
#parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/tech/content writer/JayaaSProfile.pdf")
#parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/tech/software/Pierre-HenriRolandProfile.pdf")
#parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/tech/others/Alfina_Muthu_Chennai_1.05_yrs.docx")
contents=[clean_doc(parsedPDF['content'])]
            #print (jd_contents)
print(contents)

['arju utpal 1112 1st cross 4th main ns palya btm 2nd stage560076  c 9036248907  arjuutpalgmailcom career objective part dynamic environment looking professional growth way harmonizing organizational goals personal goals educational qualification be cmr institute technology bangalore visvesvaraya technological university 55 telecommunication engineering 2016 class xii college commerce patna 60 bseb 2010 class x jesus mary academy darbhanga 67 cbse 2008 projects undertaken college powerline communication plc mini project quiz software class project smart home technology using kinect camera help ann final year project technical skills rhcsa red hat rhel 70 going certification course system administration good knowledge windows linux operating system good overview unix kernel architecture good understanding osi model tcpip protocol suite lanwan vpn virtual machine router switch hub dhcp troubleshooting personal computers basic network components familiar applications like ms office suite 

In [7]:
def prpInput(pp):
    tx = Tokenizer()
    tx.fit_on_texts(pp)
    vocab_size = len(tx.word_index)+1
    encoded_docs = tx.texts_to_sequences(pp)
    max_length = 1000
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return padded_docs

In [8]:
mat=prpInput(contents)
mat

array([[ 20,  21,  22,  23,   5,  24,  25,  26,  27,  28,  29,  30,  31,
         32,   6,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
          7,   8,   7,  44,  45,  46,  47,  48,   9,  49,  50,  51,  52,
         53,  54,  55,  56,   1,  57,   2,  58,  59,  60,  61,  62,   1,
         63,  10,  11,  64,  65,  66,  67,  12,  68,  69,   2,  70,  71,
         72,  73,   3,  13,  74,   1,   3,  75,  76,   9,  77,  78,  79,
         80,  81,  82,  83,   3,  84,  85,  86,  14,  87,  88,  89,  90,
         15,  91,  16,  92,   4,  93,  94,  95,  96,  16,   4,  97,  98,
         99, 100,   4, 101, 102, 103, 104, 105,  17, 106, 107, 108, 109,
        110, 111, 112, 113, 114,   8, 115, 116, 117, 118, 119, 120, 121,
        122, 123,  17, 124, 125, 126, 127, 128, 129,  13, 130, 131, 132,
        133,  18, 134,   3,  15,   2, 135, 136, 137,   2, 138, 139, 140,
         19, 141, 142, 143,   1,  18,   4, 144, 145,  10,  11,  19,  12,
        146,   6, 147, 148, 149, 150, 151, 152, 153

In [271]:
prd = model.predict(mat)


In [272]:
labels_index = {'IT':0 ,'contentwriter':1, 'other':2, 'software':3}
rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)
def result(prd,contents):
    y_classes = prd.argmax(axis=-1)
    print("num. of classes:", len(y_classes))
    lx=[]
    for idx,lb in enumerate(y_classes):
        lx.append([contents[idx],rev_lable_index[lb]])
    return lx

{0: 'IT', 1: 'contentwriter', 2: 'other', 3: 'software'}


In [273]:
result(prd,contents)

num. of classes: 1


[['arju utpal 1112 1st cross 4th main ns palya btm 2nd stage560076  c 9036248907  arjuutpalgmailcom career objective part dynamic environment looking professional growth way harmonizing organizational goals personal goals educational qualification be cmr institute technology bangalore visvesvaraya technological university 55 telecommunication engineering 2016 class xii college commerce patna 60 bseb 2010 class x jesus mary academy darbhanga 67 cbse 2008 projects undertaken college powerline communication plc mini project quiz software class project smart home technology using kinect camera help ann final year project technical skills rhcsa red hat rhel 70 going certification course system administration good knowledge windows linux operating system good overview unix kernel architecture good understanding osi model tcpip protocol suite lanwan vpn virtual machine router switch hub dhcp troubleshooting personal computers basic network components familiar applications like ms office suite

In [274]:
import pandas as pd
pd.DataFrame(result(prd,contents),columns=['File Content','Label'])

num. of classes: 1


,File Content,Label
0,arju utpal 1112 1st cross 4th main ns palya bt...,IT


In [307]:
def test_load():
    i=0
    test=[]
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/testmix_tech"):
        print (i, file)
        parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/testmix_tech/"+file)
        content = clean_doc(parsedPDF['content'])
        test.append(content)
        #mat=prpInput(contents)
        #prd = model.predict(mat)
        #print(result(prd,contents))
        #pd.DataFrame(result(prd,contents),columns=['File Content','Label'])
        i=i+1
        
    return test
testinput = test_load()

0 Abhilash Rajan.doc
1 arpitmandloi[5_3].docx
2 arlinjohn[2_7].docx
3 AJAY_RESUME (2).pdf
4 Abhijith .pdf
5 Ajay_Kemble_Mumbai_0.00_yrs.doc
6 alok_trivedi_Delhi_Region_3.06_yrs.docx
7 Anoop[4_0].pdf
8 BalajiDaruvoori[8_0].doc
9 Allan John -Interested.docx
10 AnoopaSebastian[2_0].doc


In [308]:
h=prpInput(testinput)

In [309]:
prdi = model.predict(h)

In [310]:
def result(prdi,testinput):
    y_classes = prdi.argmax(axis=-1)
    print("num of target items:",len(y_classes))
    lx=[]
    rev_lable_index= {0:'IT' ,1:'contentwriter', 2:'other', 3:'software'}
    for idx,lb in enumerate(y_classes):
        lx.append([testinput[idx],rev_lable_index[lb]])
    return lx

In [311]:
#result(prdi,testinput)
import pandas as pd
pd.DataFrame(result(prdi,testinput),columns=['File Content','Label'])

num of target items: 11


,File Content,Label
0,executive summary abhilash rajan padmalayam ed...,software
1,mobile 917406741419 919035921745 arpitmandloig...,contentwriter
2,divya mathewtechwriter arlin john email id arl...,contentwriter
3,ajay resume curriculum vitae ajay augustine so...,software
4,abhijithlenin vishnucottagehouse kalvamkodampo...,software
5,sumit sanghai ajay suresh kemble 9 sadguru niw...,IT
6,alok kumar trivedi mobile 91 9971082484 706557...,IT
7,r anoop mobile 91 9745372841 email anoopcinefi...,IT
8,balaji daruvoori balaji daruvoori email balaji...,contentwriter
9,allan john flat 1302 h29 awho sandeep vihar re...,IT
